# ETL - Extract, Transform, Load
Общий термин для всех процессов миграции данных из одного источника в другой (экспорт, импорт, конвертация данных; парсинг файлов, web-scrapping и пр.)

Типичные этапы ETL процесса:
- извлечение из источника данных
- очистка (приведение разнородных данных к единому формату, удаление лишнего и пр)
- обогащение 
- трансформирование 
- загрузка (интеграция в единую модель)

Сложные ETL процессы разбиваются на цепочку более простых.


Рассмотрим частный случай импорта данных из внешнего файла. Например, из Excel или csv.


### Образец файла с исходными данными:
<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/etl/source.png?raw=1" />

### Модель данных в целевой БД:
<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmingСompetitionsDB.png?raw=1" />

Проведем первичный _data profiling_, определив:
- метрики данных (список атрибутов, их типы, длину строковых полей, null/not null, потенциальный ключ, степень соответствия целевым атрибутам и пр)
- аномалии (грязные или отсутствующие данные, множественные значения и пр)
- возможный способ осуществления парсинга 

### Требования к Staging-ETL (v1.0):
<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/etl/requrements - stg_Competitions.png?raw=1" />

В атрибутах все еще присутствуют множественнные значения, мы не будем готовы загрузить такие данные в целевые таблицы (например, Фамилия+Имя, Клуб+Город, Группа+Длина дистанции+Стиль плавания).

Нужно детальное сравнение с целевыми атрибутами и уточнение требований.

### Уточненные требования:
<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/etl/requrements_upd - stg_Competitions.png?raw=1" />

Создадим стейджинговую таблицу для загрузки исходных исходных данных в сыром виде (фактически, мы будем создавать ELT а не ETL код):

In [0]:
use tempdb
go

drop table if exists dbo.stg_competitions
go

create table dbo.stg_competitions
(
	id int			 not null identity,

	F1 nvarchar(255) null,
	F2 nvarchar(255) null,
	F3 nvarchar(255) null,
	F4 nvarchar(255) null,
	F5 nvarchar(255) null,
	F6 nvarchar(255) null,
	F7 nvarchar(255) null,
	F8 nvarchar(255) null,
	F9 nvarchar(255) null,

	constraint PK_stg_Competitions primary key
	(
		id
	)
) 
go

Будем полагать исходные данные находятся в текстовом файле формата csv.<br/>
Если, к примеру, изначально данные находятся в Excel, конверсию в csv можно сделать программно или средствами самого Excel.

Грузим данные в staging-таблицу:

In [1]:
truncate table dbo.stg_Competitions

bulk insert dbo.stg_competitions 
from 'c:/temp/итоговый протокол.csv'
   with (
      format = 'csv',
	  codepage = 65001,	 
	  formatfile = 'c:/temp/stg_competitions.fmt',
	  --firstrow = 1,
      fieldterminator = ',' 
)
go

select top 5 * from dbo.stg_Competitions

Commands completed successfully.

Total execution time: 00:00:00.0429511

(1470 rows affected)

Total execution time: 00:00:00.0366606

(5 rows affected)

Total execution time: 00:00:00.0711107

id,F1,F2,F3,F4,F5,F6,F7,F8,F9
1,NULL,ПРОТОКОЛ,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,NULL,"Международного турнира по плаванию ""Весенние старты"",",NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,NULL,"по программе, ""Кубок ""Надежды-2019""",NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,NULL,"г.Жлобин, Беларусь,22-24.04.2019, бассейн 50м.",NULL,NULL,NULL,NULL,NULL,NULL,NULL
5,NULL,1 день - 22.04.2019,NULL,NULL,NULL,NULL,NULL,NULL,NULL


Альтернативно, ту же задачу можно было сделать с помощью мастера импорта данных или с помощью 
- ```openrowset```:

```sql
        select *
        from openrowset(
            bulk 'c:/temp/итоговый протокол.csv', 
            formatfile = 'c:/temp/stg_competitions.fmt', 
            format='csv', 
            codepage='65001'
        ) a  
```

- bcp:
```bash
        bcp testdb.dbo.stg_Competitions format nul -f ./stg_Competitions.fmt -S 127.0.0.1 -U SA -P "password"

        bcp stg_Competitions in /home/gavrilenkotimofey/Соревнования.csv -f ./stg_Competitions.fmt -S localhost -U sa -P "password" -d testdb 
```

### Попробуем решить задачу одним запросом ``` SELECT ```, разбив его на ряд CTE-модулей

In [4]:
--Шаг 1. Попытка разбить данные на группы (одна группа на одну соревновательную дисциплину): 

;with cte_list_of_group_discipline
as
(
    select c.id, 
         c.F2 as group_discipline
    from dbo.stg_Competitions c   
    join dbo.stg_Competitions c2 on c.id+2 = c2.id
    where c2.F1 = '1' 
)
select * from cte_list_of_group_discipline

Commands completed successfully.

Total execution time: 00:00:00.0017419

(30 rows affected)

Total execution time: 00:00:00.0119894

id,group_discipline
6,Девочки 2007г.р. и моложе - 50 баттерфляй
60,Мальчики 2007г.р. и моложе - 50 баттерфляй
150,Девочки 2006 г.р. - 100 на спине
187,Мальчики 2006 г.р. - 100 на спине
251,Девушки 2004-2005 г.р. - 400 вольный стиль
275,Юноши 2004-2005 г.р. - 400 вольный стиль
345,Девочки 2007г.р. и моложе -50 вольный стиль
400,Мальчики 2007г.р. и моложе -50 вольный стиль
492,Девочки 2006г.р. - 100 вольный стиль
531,Мальчики 2006г.р. - 100 вольный стиль


Внимательно проанализировав полученный результат, мы находим признак грязных данных (Будник Виктория в названии группы).
После выяснения причины, оказывается, в исходных данных нарушен общий порядок Название группы-Строка заголовка-Данные.

Нужна доработка кода выше под эту особенность.

### Предварительный анализ "сырых" данных
<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/etl/source-analysis.png?raw=1" />

Определение соревновательных групп:

In [5]:
;with cte_list_of_group_discipline
as
(
    select c.id, 
         c.F2 as group_discipline
    from dbo.stg_Competitions c
    join dbo.stg_Competitions c1 on c.id+1 = c1.id
    join dbo.stg_Competitions c2 on c.id+2 = c2.id
    where (c1.F1 = '1' and /*(charindex('-', c.F2)<>0))*/ c.F3 is null) or 
          (c2.F1 = '1' and /*(charindex('-', c.F2)<>0))*/ c.F3 is null)
)
select * from cte_list_of_group_discipline

(30 rows affected)

Total execution time: 00:00:00.0191846

id,group_discipline
6,Девочки 2007г.р. и моложе - 50 баттерфляй
60,Мальчики 2007г.р. и моложе - 50 баттерфляй
150,Девочки 2006 г.р. - 100 на спине
187,Мальчики 2006 г.р. - 100 на спине
251,Девушки 2004-2005 г.р. - 400 вольный стиль
275,Юноши 2004-2005 г.р. - 400 вольный стиль
345,Девочки 2007г.р. и моложе -50 вольный стиль
400,Мальчики 2007г.р. и моложе -50 вольный стиль
492,Девочки 2006г.р. - 100 вольный стиль
531,Мальчики 2006г.р. - 100 вольный стиль


Определение места и времени проведения соревнования:

In [9]:
;with cte_list_of_competition_days
as
(
    select c.id, 
         c.F2 as group_discipline
    from dbo.stg_Competitions c    
    join dbo.stg_Competitions c3 on c.id+3 = c3.id
    where c3.F1 = '1' and (charindex('-', c.F2)<>0)
)
select * from cte_list_of_competition_days

(2 rows affected)

Total execution time: 00:00:00.0163952

id,group_discipline
5,1 день - 22.04.2019
597,2 день - 23.04.2019


"Разворот" извлеченных атрибутов в столбцы:

In [2]:
;with cte_trim_fields
as
(
    select ltrim(rtrim(F1)) as F1,
	       ltrim(rtrim(F2)) as F2,
		   ltrim(rtrim(F3)) as F3,
		   ltrim(rtrim(F4)) as F4,
		   ltrim(rtrim(F5)) as F5,
		   ltrim(rtrim(F6)) as F6,
		   ltrim(rtrim(F7)) as F7,
		   id
	from stg_Competitions
),
cte_list_of_group_discipline
as
(
    select c.id, 
         c.F2 as group_discipline
    from cte_trim_fields c
    join cte_trim_fields c1 on c.id+1 = c1.id
    join cte_trim_fields c2 on c.id+2 = c2.id
    where (c1.F1 = '1' and (charindex('-', c.F2)<>0)) or 
          (c2.F1 = '1' and (charindex('-', c.F2)<>0))
),
cte_list_of_ranges
as
(
    select id start_id, 
	       lead(id) over(order by id) end_id
    from cte_list_of_group_discipline
),
cte_pivot
as
(
	select c.*, gd.group_discipline
	from cte_trim_fields c
	join cte_list_of_ranges rr on c.id between rr.start_id and case when rr.end_id-1 is not null then rr.end_id-1 else (select count(1) from stg_Competitions) end
	join cte_list_of_group_discipline gd on gd.id = rr.start_id
)
select top 100 * from cte_pivot

(100 rows affected)

Total execution time: 00:00:00.0398516

F1,F2,F3,F4,F5,F6,F7,id,group_discipline
NULL,Девочки 2007г.р. и моложе - 50 баттерфляй,NULL,NULL,NULL,NULL,NULL,6,Девочки 2007г.р. и моложе - 50 баттерфляй
NULL,Ф.И. спортсмена,Год рожд.,"Город, ДЮСШ",Страна,Результат,Очки,7,Девочки 2007г.р. и моложе - 50 баттерфляй
1,Евмененко София,07,"Долгопрудный,ДЮСШ",RUS,33.5,387,8,Девочки 2007г.р. и моложе - 50 баттерфляй
2,Редюк Алина,07,"Гомель-1,ГОЦОР ввс",BLR,35.44,327,9,Девочки 2007г.р. и моложе - 50 баттерфляй
3,Козловская Кристина,07,"Минск,""Волна""",BLR,36.1,309,10,Девочки 2007г.р. и моложе - 50 баттерфляй
4,Карпочева Екатерина,07,"Речица,Олимп",BLR,36.62,296,11,Девочки 2007г.р. и моложе - 50 баттерфляй
5,Гисак Анна,07,"Могилев,СДЮШОР",BLR,36.96,288,12,Девочки 2007г.р. и моложе - 50 баттерфляй
6,Козлова Александра,08,"Речица,Олимп",BLR,36.96,288,13,Девочки 2007г.р. и моложе - 50 баттерфляй
7,Кондратович Вероника,07,"Минск,""Волна""",BLR,38.22,261,14,Девочки 2007г.р. и моложе - 50 баттерфляй
8,Оргиш Ксения,07,"Бобруйск,СДЮШОР",BLR,38.25,260,15,Девочки 2007г.р. и моложе - 50 баттерфляй


Мы приблизились к выполнению требований, теперь сконцентрируемся на разбиении множественных атрибутов на атомарные значения.

Попробуем разбить Фамилию Имя:

In [3]:
declare @name varchar(100) = N'Евмененко София'

select @name as full_name,
       iif(charindex(' ', @name)<>0, left(@name, charindex(' ', @name) - 1), @name) as last_name,
       iif(charindex(' ', @name)<>0, right(@name, len(@name) - charindex(' ', @name)), null) as first_name

(1 row affected)

Total execution time: 00:00:00.0097392

full_name,last_name,first_name
Евмененко София,Евмененко,София


Проверим корректно ли происходит разбиение на всем множестве записей (у всех таких записей значение в колонке F1 не пусто):

In [7]:
-- результаты на первый взгляд положительны. 
--select     F2,
--           iif(charindex(' ', F2)<>0, left(F2, charindex(' ', F2) - 1), F2) as last_name,
--           iif(charindex(' ', F2)<>0, right(F2, len(F2) - charindex(' ', F2)), null) as first_name
--from dbo.stg_Competitions   
--where F1 is not null        

-- но лучше убедиться убедиться лишний раз, потратив немного больше времени...
select distinct iif(charindex(' ', F2)<>0, left(F2, charindex(' ', F2) - 1), F2) as last_name		   
from dbo.stg_Competitions   
where F1 is not null        

select distinct iif(charindex(' ', F2)<>0, right(F2, len(F2) - charindex(' ', F2)), null) as first_name
from dbo.stg_Competitions   
where F1 is not null        


(324 rows affected)

(106 rows affected)

Total execution time: 00:00:00.0392608

last_name
Абрамцева
Авдей
Авдейчик
Авраменко
Акудович
Акульков
Алексеев
Алесенко
Алещенко
Ананченко


first_name
Антон
Дарья
Иван
Александр
Александра
Алексей
Алеся
Алина
Амина
Анастасия


Время на data profiling было потрачено не зря!

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/etl/baddata-firstname.png?raw=1" />

В поле ```first_name``` нужно взять лишь первое слово (если их несколько):


In [8]:
select  left(first_name, len(first_name) - charindex(' ', reverse(first_name))) first_name
from (
    select distinct iif(charindex(' ', F2)<>0, right(F2, len(F2) - charindex(' ', F2)), null) as first_name
    from dbo.stg_Competitions   
    where F1 is not null   
) q

(106 rows affected)

Total execution time: 00:00:00.0142753

first_name
""
""
""
Александр
Александра
Алексей
Алеся
Алина
Амина
Анастасия


Сделаем подобные проверки для каждого поля. Подкорректируем формулы извлечения атрибутов для всех имеющихся issue.

Заметим, помимо прочего, что у всех атрибутов могут быть пробелы как До, так и После их значения. Будем применять ```ltrim(rtrim(value))```.

### Все это - data profiling и меры по очистке данных!
Очевидно, мы не можем исправить все возможные проблемы. В таком случае, часть данных будет загружена с ошибками.

### Высокий процент ошибок парсера говорит как о плохом качестве исходных данных, так и о плохой работе ETL-специалиста в части его подготовительной(исследовательской) работы перед написанием ETL.


Еще одна проблема - результаты заплывов (время). Строки не находятся в едином формате и не конвертируются в тип ```time(2)```.

Примеры значений: ```'44.1'; '42,35'; 'DSQ'; '1.09,98', '59'```. Среди значений времени присутствуют строки.

Будем разбивать значения этого поля на результаты-время и результаты-признак дисквалификации. При этом само результаты-время разложим на компоненты hh-mm-ss-ms с намерением впоследствии применить функцию ```timefromparts(hh, mm, ss, ms, precision)```

In [1]:
declare @result varchar(100) = ''

--рекурсивная cte для разбиения строки-времени на отдельные составляющие-"тоукены"
;with cte_parse_time
as
(
    select left(result, len(result) - patindex('%[:,.]%', reverse(result))) left_to_parse, 
	       try_parse(right(result, patindex('%[:,.]%', reverse(result)) - 1) as int) token,
		   it = 1,
		   id
    from (values(1, '44.1'), (2, '42,35'), (3, '1.09,98')) query(id, result)
    union all
    select iif(patindex('%[:,.]%', reverse(left_to_parse)) <> 0, left(left_to_parse, len(left_to_parse) - patindex('%[:,.]%', reverse(left_to_parse))), '0') left_to_parse, 
	       try_parse(iif(patindex('%[:,.]%', reverse(left_to_parse)) <> 0, right(left_to_parse, patindex('%[:,.]%', reverse(left_to_parse)) - 1), left_to_parse) as int) token,
		   it = it + 1,
		   id	   
    from cte_parse_time
	where it < 4 	
)
select * from cte_parse_time order by id, it

(12 rows affected)

Total execution time: 00:00:00.1411529

left_to_parse,token,it,id
44,1,1,1
0,44,2,1
0,0,3,1
0,0,4,1
42,35,1,2
0,42,2,2
0,0,3,2
0,0,4,2
1.09,98,1,3
1,9,2,3


Теперь соберем все вместе.

### Окончательный вариант:

In [10]:
;with cte_trim_fields
as
(
    select ltrim(rtrim(F1)) as F1,
	       ltrim(rtrim(F2)) as F2,
		   ltrim(rtrim(F3)) as F3,
		   ltrim(rtrim(F4)) as F4,
		   ltrim(rtrim(F5)) as F5,
		   ltrim(rtrim(F6)) as F6,
		   ltrim(rtrim(F7)) as F7,
		   id
	from stg_Competitions
),
cte_list_of_competition_days
as
(
    select c.id, 
           c.F2 as competition_day
    from dbo.stg_Competitions c    
    join dbo.stg_Competitions c3 on c.id+3 = c3.id
    where c3.F1 = '1' and (charindex('-', c.F2)<>0)
),
cte_list_of_group_discipline
as
(
    select c.id, 
           c.F2 as group_discipline
    from cte_trim_fields c
    join cte_trim_fields c1 on c.id+1 = c1.id
    join cte_trim_fields c2 on c.id+2 = c2.id
    where (c1.F1 = '1' and (charindex('-', c.F2)<>0)) or 
          (c2.F1 = '1' and (charindex('-', c.F2)<>0))
),
cte_list_of_ranges_competition_days
as
(
    select id start_id, 
	       lead(id) over(order by id) end_id
    from cte_list_of_competition_days
),
cte_list_of_ranges_discipline
as
(
    select id start_id, 
	       lead(id) over(order by id) end_id
    from cte_list_of_group_discipline
),
cte_pivot
as
(
	select c.*, gd.group_discipline, cd.competition_day
	from cte_trim_fields c
	join cte_list_of_ranges_discipline rd        on c.id between rd.start_id and case when rd.end_id-1 is not null then rd.end_id-1 else (select count(1) from dbo.stg_Competitions) end
	join cte_list_of_group_discipline gd         on gd.id = rd.start_id
	join cte_list_of_ranges_competition_days rcd on c.id between rcd.start_id and case when rcd.end_id-1 is not null then rcd.end_id-1 else (select count(1) from dbo.stg_Competitions) end
	join cte_list_of_competition_days cd         on cd.id = rcd.start_id 
),
cte_transform
as
(
	select 
	       F1 as place,

		   iif(charindex(' ', F2)<>0, left(F2, charindex(' ', F2) - 1), F2) as last_name,
		   iif(charindex(' ', F2)<>0, right(F2, len(F2) - charindex(' ', F2)), null) as first_name,

		   iif(len(F3)=2, iif(left(F3, 1) in ('8','9'), '19'+F3,'20'+F3), F3) as birth_year,

		   iif(charindex(',', F4)<>0, left(F4, charindex(',', F4) - 1), F4) as city,
		   iif(charindex(',', F4)<>0, right(F4, len(F4) - charindex(',', F4)), null) as team,
		   
		   F5 as country,	 
		  
		   iif(charindex('D', F6)=0, iif(len(F6)=2, F6+'.00', F6), null) as result,
		   iif(charindex('D', F6)<>0, F6, null) as disc,
		   
		   F7 as points,
   		   
		   rtrim(left(group_discipline, len(group_discipline) - charindex('-', reverse(group_discipline)))) as athlete_group,
		   ltrim(right(group_discipline, charindex('-', reverse(group_discipline)) - 1)) as discipline,
		   
		   ltrim(right(competition_day, len(competition_day) - charindex('-', competition_day))) as [date],

		   id
	from cte_pivot
	where F1 is not null
),
cte_parse_time
as
(
    select left(result, len(result) - patindex('%[:,.]%', reverse(result))) left_to_parse, 
	       try_parse(right(result, patindex('%[:,.]%', reverse(result)) - 1) as int) token,
		   it = 1,
		   id
    from cte_transform
    union all
    select iif(patindex('%[:,.]%', reverse(left_to_parse)) <> 0, left(left_to_parse, len(left_to_parse) - patindex('%[:,.]%', reverse(left_to_parse))), '0') left_to_parse, 
	       try_parse(iif(patindex('%[:,.]%', reverse(left_to_parse)) <> 0, right(left_to_parse, patindex('%[:,.]%', reverse(left_to_parse)) - 1), left_to_parse) as int) token,
		   it = it + 1,
		   id
    from cte_parse_time
	where it < 4 	
), 
cte_clean_and_format
as
(
    select try_parse(place as int) place,
		   left(last_name, len(last_name) - charindex(' ', reverse(last_name))) last_name,
	       left(first_name, len(first_name) - charindex(' ', reverse(first_name))) first_name,
		   try_parse(birth_year as int) birth_year,
		   city,
		   replace(team, '"', '') team,
		   country,
		   timefromparts(h.token, m.token, s.token, ms.token, 2) result,
		   disc,
		   try_parse(points as int) points,		   
		   athlete_group,			   
		   try_parse(left(discipline, charindex(' ', discipline)) as int) distance,
	       right(discipline, len(discipline) - charindex(' ', discipline) ) as style,
		   try_convert(date, [date], 104) [date],
		   t.id
	from cte_transform t
	join cte_parse_time h on t.id = h.id and h.it = 4
	join cte_parse_time m on t.id = m.id and m.it = 3
	join cte_parse_time s on t.id = s.id and s.it = 2
	join cte_parse_time ms on t.id = ms.id and ms.it = 1
)
select top 50 * from cte_clean_and_format


(50 rows affected)

Total execution time: 00:00:03.2461838

place,last_name,first_name,birth_year,city,team,country,result,disc,points,athlete_group,distance,style,date,id
41,Диченков,Сергей,2006,Гомель,Гомсельмаш,BLR,00:03:06.0900000,NULL,229,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1468
37,Буканов,Артем,2006,Гомель,СДЮШОР 7,BLR,00:03:03.9200000,NULL,238,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1464
33,Осипенко,Георгий,2006,Гомель-1,ГОЦОР ввс,BLR,00:03:01.8100000,NULL,246,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1460
29,Чирейко,Кирилл,2006,Речица,СДЮШОР,BLR,00:03:00.3900000,NULL,252,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1456
25,Карпеко,Константин,2006,Речица,Олимп,BLR,00:02:57.1300000,NULL,266,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1452
21,Чичкан,Александр,2006,Минск,Янтарь,BLR,00:02:53.2500000,NULL,284,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1448
42,Лагуновский,Стас,2006,Минск,Волна,BLR,00:03:06.5100000,NULL,228,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1469
43,Марченко,Руслан,2006,Гомель,ДЮСШ-6,BLR,00:03:07.1000000,NULL,226,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1470
39,Шидловский,Никита,2006,Минск,Волна,BLR,00:03:04.4600000,NULL,236,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1466
35,Утёмов,Станислав,2006,Гомель-1,ГОЦОР ввс,BLR,00:03:02.2200000,NULL,244,Мальчики 2006г.р.,200,комплексное плавание,2019-04-23,1462


### Используя язык SQL, мы создали основу ELT-процесса парсинга полуструктурированных данных.

В будущем, этот код можно оформить как хранимую процедуру или табличную функцию и использовать как отдельный компонент в "большом" ETL-процессе